<a href="https://colab.research.google.com/github/ge43jef/GEEHYDRO/blob/block2/gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download NDVI Image Collection

Authenticate and Initialize Earth Engine

In [ ]:
import ee
# Trigger the authentication flow
ee.Authenticate()

# Initialize the library
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=lz5vAC92S3bPmQZCQoOy7W8MjKza0yRhVjONpMKPgIY&tc=eqzdcHW_uCZHY5bcsjuK-9NmpGj7rb7zsDtiOYoHujc&cc=KrGl-e8KZfh8YaASG7i2x8ZiRC9Jv0OGMaI7xq99s4s

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VPVYr7S5Xt1MmQMSaWA8nTtxlm76r992YgdHLdXYcqYJHkD0rxAi64

Successfully saved authorization token.


EEException: ignored

In [ ]:
# Creat a collection
geometry = ee.Geometry.Point([31.5051 , -26.6061]) # location of data, lat lon information

location = geometry.coordinates().getInfo()[: : -1] # Invert location information

s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") # imagecollection ID : https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED
rgbVis = {
    'min':0.0,
    'max':3000,
    'bands':['B4','B3','B2'],
} # parameters for the visualize

# Write a function for cloud masking
def mask2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).select('B.*').copyProperties(image,['system:time_start'])

# s2.filter(date).filter(cloud contamination).filter(location).map(every image apply the mask2clouds)
filtered = s2.filter(ee.Filter.date('2020-02-02' , '2020-02-02'))\
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE' , 30))\
        .filter(ee.Filter.bounds(geometry)).map(mask2clouds)

# Function computes NDVI for an image and add it as a band
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5' , 'B4']).rename('ndvi')
  return image.addBands(ndvi)

withNdvi = filtered.map(addNDVI)
median = withNdvi.median()



EEException: ignored

In [ ]:
image_ids = withNdvi.aggregate_array('system:index').getInfo()
print('Total images:' , len(image_ids))

In [ ]:
# Export with 100m resolution
for i , image_id in enumerate(image_ids):
  image = ee.Image(withNdvi.filter(ee.Filter.eq('system:index',image_id)).first())
  task = ee.batch.Export.image.toDrive(** {
      'image':image.select('ndvi'),
      'description': 'Image Export {}'.format(i+1),
      'fileNamePrefix':image.id().getInfo(),
      'folder':'Example_folder',
      'scale':100,
      'region': image.geometry().bounds().getInfo()['coordinates'],
      'maxPixels': 1e10

  })
  task.start()
  print('Start Task:' , i + 1)
